In [2]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import matplotlib.pyplot as plt # 데이터 시각화

from datetime import datetime, timedelta # 시간 데이터 처리
from statsmodels.tsa.arima_model import ARIMA # ARIMA 모델
%matplotlib inline

In [12]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("submission_1002.csv")

In [13]:
def to_panel_format(df):
    
    place_id=[]; time=[] ; target=[] # 빈 리스트를 생성합니다.
    
    df['Time'] = pd.to_datetime(df['Time']) 
    df = df.set_index('Time')
    
    for i in df.columns:
        for j in range(len(df)):
            place_id.append(i) # place_id에 미터 ID를 정리합니다.
            time.append(df.index[j]) # time에 시간대를 정리합니다.
            target.append(df[i].iloc[j]) # target에 전력량을 정리합니다.

    ret = pd.DataFrame({'place_id':place_id,'time':time,'target':target})
    ret = ret.dropna() # 결측치를 제거합니다.
    ret = ret.set_index('time') # time을 인덱스로 저장합니다.
    return ret

In [14]:
test = to_panel_format(test)

In [15]:
test.head()

,place_id,target
time,,
2018-04-18 19:00:00,X26,2.527
2018-04-18 20:00:00,X26,2.641
2018-04-18 21:00:00,X26,2.368
2018-04-18 22:00:00,X26,1.899
2018-04-18 23:00:00,X26,1.956


In [18]:
test.loc[test['place_id'] == 'X26']

,place_id,target
time,,
2018-04-18 19:00:00,X26,2.527
2018-04-18 20:00:00,X26,2.641
2018-04-18 21:00:00,X26,2.368
2018-04-18 22:00:00,X26,1.899
2018-04-18 23:00:00,X26,1.956
...,...,...
2018-06-30 19:00:00,X26,1.255
2018-06-30 20:00:00,X26,1.596
2018-06-30 21:00:00,X26,1.379


In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))

def get_optimal_params(y):
    param_dict = {}
    for param in pdq:
        try:
            model = ARIMA(y, order=param)
            results_ARIMA = model.fit(disp=-1)
            param_dict[results_ARIMA.aic] = param
        except:
            continue

    min_aic = min(param_dict.keys())
    optimal_params = param_dict[min_aic]
    return optimal_params

In [ ]:
agg={}
for key in df['place_id'].unique(): # 미터ID 200개의 리스트를 unique()함수를 통해 추출합니다.
    temp = df.loc[df['place_id']==key] # 미터ID 하나를 할당합니다.
    temp_1h = temp.resample('1h').sum() # 1시간 단위로 정리합니다.
    temp_1day = temp.resample('D').sum() # 1일 단위로 정리합니다.

    # 시간별 예측
    model = ARIMA(temp_1h['target'], order=get_optimal_params(temp_1h['target'])) # AIC를 최소화하는 최적의 파라미터로 모델링합니다.
    results_ARIMA = model.fit(disp=-1)
    fcst = results_ARIMA.forecast(24) # 24시간을 예측합니다.

    sub = pd.DataFrame() # a라는 데이터프레임에 예측값을 정리합니다.
    
    for i in range(24):
        sub['X2018_7_1_'+str(i+1)+'h'] = [fcst[0][i]] # column명을 submission 형태에 맞게 지정합니다.

        
    # 일별 예측
    model = ARIMA(temp_1day['target'], order=get_optimal_params(temp_1day['target'])) # AIC를 최소화하는 최적의 파라미터로 모델링합니다.
    results_ARIMA = model.fit(disp=-1)
    fcst = results_ARIMA.forecast(10) # 10일을 예측합니다.

    for i in range(10):
        sub['X2018_7_'+str(i+1)+'_d'] = [fcst[0][i]] # column명을 submission 형태에 맞게 지정합니다.
    
    
    # 월별 예측
    # 일별로 예측하여 7월 ~ 11월의 일 수에 맞게 나누어 합산합니다.
    fcst = results_ARIMA.forecast(153)
    sub['X2018_7_m'] = [np.sum(fcst[0][:31])] # 7월 
    sub['X2018_8_m'] = [np.sum(fcst[0][31:62])] # 8월
    sub['X2018_9_m'] = [np.sum(fcst[0][62:92])] # 9월
    sub['X2018_10_m'] = [np.sum(fcst[0][92:123])] # 10월
    sub['X2018_11_m'] = [np.sum(fcst[0][123:153])] # 11월
    
    sub['meter_id'] = key 
    agg[key] = sub[submission.columns.tolist()]
    print(key)
print('---- Modeling Done ----')